# Libs & Settings

In [1]:
import re
import sys
import time
import locale
import warnings
import numpy as np
import pandas as pd
from ncls import NCLS
from pathlib import Path
from tqdm.auto import tqdm
from functools import reduce
from bs4 import BeautifulSoup
from datetime import datetime
from typing import Tuple, List, Optional
from sentence_transformers import SentenceTransformer, util

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Import developer modules.
module_path = str(Path.cwd().parent / "utils")
if module_path not in sys.path:
    sys.path.append(module_path)

from processing import (
    process_time,
    count_months,
    parse_resume,
    hh_job_preparation,
    get_job_rank
)

In [4]:
# Define data folder.
data_folder = Path().cwd().parent / 'data'

# Parsing Data

In [7]:
# File path generator.
files = data_folder.glob('*.html')
# Collect jobs from all resumes.
df = reduce(
    # Concatenate all dataframes together.
    lambda x, y: pd.concat([x, y]),
    # Generate a list of dataframes with jobs.
    [
        pd.DataFrame(
            data=parse_resume(
                file=file, 
                user_id=id,
                name_process_func=hh_job_preparation # put here string processing function
            ),
            columns=['user_id', 'month_cnt', 'start_date', 'end_date', 'job_name', 'job_desc']
        ) for id, file in enumerate(tqdm(list(files), 'Read and process resume'))
    ]
)

Read and process resume:   0%|          | 0/1001 [00:00<?, ?it/s]

In [10]:
# Save data.
# df.to_pickle('parsed.pickle')

# Process Data

In [5]:
# Load data.
# df = pd.read_pickle('parsed.pickle')

In [8]:
df_rank = get_job_rank(df=df)

Processing users:   0%|          | 0/1001 [00:00<?, ?it/s]

In [9]:
df_rank.head()

,user_id,month_cnt,start_date,end_date,job_name,job_desc,job_level_simple,job_level_intersect
0,0,68,2018-04-01,-1,специалист сервисно-монтажной службы,"Сборка, ремонт устройств селскохозяйственного ...",9,7
1,0,8,2017-09-01,2018-04-01,менеджер,"Совершение холодных звонков, поиск клиентов, з...",8,6
2,0,3,2016-08-01,2016-10-01,продавец-консультант,Консультирование и обслуживание покупателей.\n...,7,5
3,0,12,2015-08-01,2016-07-01,менеджер товароучета и хранения,"Работа с клиентами, обработка заявок в програм...",6,4
4,0,6,2013-11-01,2014-04-01,заведующий складом в магазине одежды,"Руководство работой склада по приему, хранению...",5,3


# Clusterization

In [12]:
# Load model.
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.45k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

In [14]:
# Select level1 job names.
corpus_sentences = list(df_rank[df_rank.job_level_intersect == 1].job_name)
# Apply encoding.
print("Encode the corpus. This might take a while")
corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus. This might take a while


Batches:   0%|          | 0/18 [00:00<?, ?it/s]

In [15]:
# Get clusters
print("Start clustering")
start_time = time.time()
#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 25 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(corpus_embeddings, min_community_size=25, threshold=0.75)
print("Clustering done after {:.2f} sec".format(time.time() - start_time))

#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", corpus_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", corpus_sentences[sentence_id])

Start clustering
Clustering done after 0.29 sec

Cluster 1, #143 Elements 
	 главный ветеринарный врач
	 ветеринарный врач
	 ветеринарный фельдшер
	 ...
	 ветеринарный фельдшер
	 ветеринарный врач
	 главный ветеринарный врач

Cluster 2, #119 Elements 
	 механик
	 главный инженер
	 инженер
	 ...
	 инженер-механик
	 механик
	 инженер-технолог

Cluster 3, #100 Elements 
	 агроном
	 агроном
	 главный агроном
	 ...
	 главный агроном
	 агроном
	 агроном

Cluster 4, #54 Elements 
	 тракторист-машинист
	 тракторист-машинист
	 тракторист-машинист
	 ...
	 машинист бульдозерист
	 тракторист-машинист
	 тракторист-машинист

Cluster 5, #53 Elements 
	 менеджер
	 начальник отдела
	 директор
	 ...
	 заместитель директора
	 директор
	 менеджер

Cluster 6, #48 Elements 
	 главный зоотехник
	 главный зоотехник
	 главный зоотехник-селекционер
	 ...
	 зоотехник
	 зоотехник
	 главный зоотехник-селекционер

Cluster 7, #43 Elements 
	 механик-водитель
	 главный механик
	 механизатор
	 ...
	 технолог
	 водител

# References

1. [Sentence Transformers](https://github.com/UKPLab/sentence-transformers): Multilingual Sentence, Paragraph, and Image Embeddings using BERT & Co.
2. [Fast clustering](https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/clustering/fast_clustering.py) implementation.